# Accu.Tuning Tutorial 3 - Multiclass Classification with Big Dataset
-----
**Accu.Tuning**을 이용하여 해결할 수 있는 여러 문제 유형 중 분류, 그 중에서도 분류 유형이 3가지 이상 있는 다중 분류(Multiclass Classification)를 시행합니다. 해당 예제에서는 특히 **Accu.Tuning** 상에서 크기가 큰 데이터를 어떻게 보다 효율적으로 처리함으로써 다중 구분 문제를 해결하는지를 소개합니다.

## 1. 데이터 준비
-----

In [1]:
import os
import numpy as np
import pandas as pd

사용할 데이터의 형태는 다음과 같습니다. 목표 변수는 5가지로 구분되는 'point_granularity' 컬럼입니다. 약 1000만 행의 데이터로 용량은 1GB입니다.

In [2]:
df = pd.read_csv('resources/1gg.csv')
df.head(10)

,policyID,statecode,county,eq_site_limit,hu_site_limit,fl_site_limit,fr_site_limit,tiv_2011,tiv_2012,eq_site_deductible,hu_site_deductible,fl_site_deductible,fr_site_deductible,point_latitude,point_longitude,line,construction,point_granularity
0,119736,FL,CLAY COUNTY,498960.0,498960.00,498960.0,498960.0,498960.00,792148.90,0.0,9979.2,0.0,0,30.102261,-81.711777,Residential,Masonry,1
1,448094,FL,CLAY COUNTY,1322376.3,1322376.30,1322376.3,1322376.3,1322376.30,1438163.57,0.0,0.0,0.0,0,30.063936,-81.707664,Residential,Masonry,3
2,206893,FL,CLAY COUNTY,190724.4,190724.40,190724.4,190724.4,190724.40,192476.78,0.0,0.0,0.0,0,30.089579,-81.700455,Residential,Wood,1
3,333743,FL,CLAY COUNTY,0.0,79520.76,0.0,0.0,79520.76,86854.48,0.0,0.0,0.0,0,30.063236,-81.707703,Residential,Wood,3
4,172534,FL,CLAY COUNTY,0.0,254281.50,0.0,254281.5,254281.50,246144.49,0.0,0.0,0.0,0,30.060614,-81.702675,Residential,Wood,1
5,785275,FL,CLAY COUNTY,0.0,515035.62,0.0,0.0,515035.62,884419.17,0.0,0.0,0.0,0,30.063236,-81.707703,Residential,Masonry,3
6,995932,FL,CLAY COUNTY,0.0,19260000.00,0.0,0.0,19260000.00,20610000.00,0.0,0.0,0.0,0,30.102226,-81.713882,Commercial,Reinforced Concrete,1
7,223488,FL,CLAY COUNTY,328500.0,328500.00,328500.0,328500.0,328500.00,348374.25,0.0,16425.0,0.0,0,30.102217,-81.707146,Residential,Wood,1
8,433512,FL,CLAY COUNTY,315000.0,315000.00,315000.0,315000.0,315000.00,265821.57,0.0,15750.0,0.0,0,30.118774,-81.704613,Residential,Wood,1
9,142071,FL,CLAY COUNTY,705600.0,705600.00,705600.0,705600.0,705600.00,1010842.56,14112.0,35280.0,0.0,0,30.100628,-81.703751,Residential,Masonry,1


In [3]:
df.shape

(9891180, 18)

## 2. 데이터 분할 및 샘플링
----
기본적으로 **Accu.Tuning**은 전체 데이터셋을 모델 학습에 사용하는 Train Set과 학습이 완료된 모델의 성능을 비교, 검증하는데 사용되는 Valid Set으로 구분하여 사용합니다. 

모델의 성능을 최종적으로 평가하는 Test Set을 따로 구성하여 사용하고 싶다면, 무작위로 추출할 Test Set 비율을 전달해줄 수 있습니다.

![test1](img/big_data_split.png)

혹은 무작위 추출이 아닌 아래와 같이 특정 열을 기준으로 추출하여 Test Set으로 활용하는 것도 가능합니다.

![test1](img/big_data_split_norandom.png)

Test Set이 구성되었을 경우, 모델을 학습 완료한 후 배포(Deploy)할 때 구성된 Test Set으로 평가된 Test Score를 확인할 수 있습니다.

![testscore](img/big_scores.png)

크기가 큰 데이터를 그대로 활용할 경우에는 제한된 시간과 자원 아래에서 효과적으로 모델 및 하이퍼파라미터를 탐색하는 것이 어렵습니다. 따라서 데이터의 크기가 클 경우 **Accu.Tuning**은 유동적으로 전체 데이터 중 일부를 샘플링하여 최적의 모델 탐색에 활용할 수 있도록 합니다. 해당 예제에서는 전체 데이터셋에서 100만건만을 활용하여 학습함으로써 모델 탐색 과정에서 보다 많은 시도를 해볼 수 있도록 합니다.

![sample](img/big_sampling.png)

## 3. 모델링 기능 활용 및 자원 관리
----

**Accu.Tuning** 상에서는 큰 데이터를 이용한 모델링에 필요한 기능들을 지원합니다. 전체 실험에 소요되는 학습 시간을 조절하거나, 불필요한 자원의 낭비를 막기 위하여 각 모델이 학습되는데 제한시간을 두거나 만일 모델 성능이 계속해서 개선되지 않는 경우 학습을 조기에 종료하는 Early Stopping 기능 등을 지원합니다. 이와 같은 기능들을 활용하여 사용자 환경과 데이터 성격, 분석 목적에 맞게 어떤 방식으로 모델을 구성할지 설정들을 변경할 수 있습니다.

<img src="img/big_model_settings.png" width="800">


또한, 사용자가 이용하는 환경 상에서 다수의 컨테이너를 활용할 수 있는 경우, **Accu.Tuning** 메인 화면에서 아래와 같이 사용 가능한 총 컨테이너 수를 확인할 수 있습니다.

![containers](img/big_container_counts.png)

그 중 해당 Experiment에 사용될 컨테이너의 개수는 아래와 같이 사용자가 지정할 수 있습니다.

![containers_option](img/big_containers.png)

**Accu.Tuning** 웹 상에서 **RUN AUTOML** 버튼을 클릭하여 바로 모델링을 시작할 수 있습니다. 컨테이너를 여러개 활용하여 학습을 할 경우, 아래 그림과 같이 각 컨테이너에 모델이 자동으로 할당되어 동시에 학습을 진행합니다.

<img src="img/big_containers_in_use.png" width="500">

## 4. 모델 활용 및 결과 예측
-----

학습이 다 이루어졌다면, 기본적으로 필요한 단일 행 (row) 수준의 예측은 모델을 따로 다운로드 받지 않아도 웹 상에서 간단하게 수행할 수 있습니다.

![prediction](img/big_prediction.png)

이 경우 예측 결과와 함께 LIME(Local Interpretable Model-Agnostic Explanation) 결과가 함께 제공되어 해당 행에 대한 모델의 로컬 해석력(Local Interpretability) 또한 파악할 수 있습니다.

![LIME](img/big_lime.png)



완료된 모델을 배포하고, Binary 형태로 불러와 **Accu.Tuning** 기능 밖에서도 활용할 수 있습니다. Leaderboard 상에서 **배포** 버튼을 클릭하면 구성된 모델을 pipeline.pkl 파일로 다운로드할 수 있습니다. 샘플을 통해 최적화된 모델 파이프라인을 그대로 활용하여 외부에서 전체 데이터셋을 학습하는 등 활용할 수 있습니다. 외부 활용을 지원하기 위하여 아래와 같이 Accu.Tuning의 도구 모음인 autoinsight_helpers 패키지를 설치해 이용할 수 있습니다.

In [4]:
!pip install \
--trusted-host github.com \
--trusted-host codeload.github.com \
https://github.com/skcc-asset-platform/autoinsight_helpers/archive/v1.0.17.zip

     \ 30 kB 1.6 MB/ss
  Created wheel for autoinsight-helpers: filename=autoinsight_helpers-1.0.15-py3-none-any.whl size=27711 sha256=85468b0105948ea70c68be2148912607197ba07664ec2aeebe930131dc2eb6f4
  Stored in directory: /tmp/pip-ephem-wheel-cache-tu2j7rvk/wheels/ca/42/49/5929a5df735753ae2910bf9602e1d18a5247a0400ebf70bdfa
Successfully built autoinsight-helpers
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [5]:
import pickle
pipeline = pickle.load(open('resources/big_pipeline.pkl', 'rb'))
print(pipeline) # CHECK PIPELINE STRUCTURE

Pipeline(steps=[('pp-0',
                 Pipeline(steps=[('integer_encode',
                                  AutoinsightIntegerEncode())])),
                ('final_estimator',
                 Pipeline(steps=[('variance_threshold', VarianceThreshold()),
                                 ['final_estimator',
                                  XGBClassifier(base_score=0.5,
                                                booster='gbtree',
                                                colsample_bylevel=1,
                                                colsample_bynode=1,
                                                colsample_bytree=1, gamma=0,
                                                gpu_id=-1,
                                                importance_type='gain',
                                                interaction_constraints='',
                                                learning_rate=1.0,
                                                max_delta_step=0, max_de

불러온 파이프라인에 샘플이 아닌 전체 데이터를 새로 학습시켜 이용할 수 있습니다.

In [ ]:
X = df.drop(['point_granularity'], axis=1)
y = df['point_granularity']

pipeline.fit(X, y)

/usr/local/lib/python3.8/site-packages/xgboost/sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:06:24] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
